In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

df = yf.download(tickers='AAPL', start='2010-01-01', end='2023-12-31', auto_adjust=False, progress=False)
df.reset_index(inplace=True)
if 'Date' in df.index.names:
    df = df.reset_index('Date')
    df.drop('Date', axis=1, inplace=True)
print("Data loaded:", df.shape)

Data loaded: (3522, 7)


In [2]:
training_size = int(len(df) * 0.8)
data_training = pd.DataFrame(df['Close'][0:training_size])

scaler = MinMaxScaler(feature_range=(0,1))
data_training_scaled = scaler.fit_transform(data_training)

x_train = []
y_train = []

for i in range(100, len(data_training_scaled)):
    x_train.append(data_training_scaled[i-100:i, 0])
    y_train.append(data_training_scaled[i, 0])

x_train = np.array(x_train)
y_train = np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

print("Training data prepared:", x_train.shape, y_train.shape)

Training data prepared: (2717, 100, 1) (2717,)


In [3]:
# Build model
model = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(100, 1)),
    Dropout(0.2),
    LSTM(units=60, return_sequences=True),
    Dropout(0.3),
    LSTM(units=80, return_sequences=True),
    Dropout(0.4),
    LSTM(units=120),
    Dropout(0.5),
    Dense(units=1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
print("Model built successfully")

Model built successfully


c:\Users\bgay9\OneDrive\Desktop\stock\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [4]:
# Train and save
model.fit(x_train, y_train, epochs=50, batch_size=32, verbose=1)
model.save('keras_model.keras')
print("Model saved successfully!")


Epoch 1/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 10s 82ms/step - loss: 0.0117
Epoch 2/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - loss: 0.0029
Epoch 3/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 85ms/step - loss: 0.0020
Epoch 4/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - loss: 0.0018
Epoch 5/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 83ms/step - loss: 0.0017
Epoch 6/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - loss: 0.0017
Epoch 7/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 83ms/step - loss: 0.0013
Epoch 8/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - loss: 0.0016
Epoch 9/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - loss: 0.0012
Epoch 10/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 82ms/step - loss: 0.0017
Epoch 11/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 84ms/step - loss: 0.0013
Epoch 12/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 85ms/step - loss: 0.0014
Epoch 13/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 86ms/step - loss: 0.0016
Epoch 14/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 81ms/step - loss: 0.0014
Epoch 15/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 7s 85ms/step - loss: 0.0015
Epo